##### Fake News Detection using LSTM - DevaSunder - 2023-08-09 11:39:00 PM

In [1]:
import tensorflow 
import pandas as pd


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
df=pd.read_csv('/kaggle/input/fake-news/train.csv')
df1=pd.read_csv('/kaggle/input/fake-news/test.csv')

In [3]:
df1.shape

(5200, 4)

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df=df.fillna('')
df1=df1.fillna('')

In [6]:
df1.shape

(5200, 4)

In [7]:
df.shape

(20800, 5)

In [8]:
X=df.drop('label',axis=1)

In [9]:
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [10]:
y=df['label']

In [11]:
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [12]:
y.shape

(20800,)

In [13]:
from tensorflow import keras

In [14]:
from keras.utils import pad_sequences
from keras.layers import Embedding
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.preprocessing.text import one_hot


In [15]:
voc_size=5000

In [16]:
# Text Preprocessing

In [17]:
from keras.layers import Embedding,Dense,LSTM
from keras.models import Sequential
from keras.utils import pad_sequences
from keras.preprocessing.text import one_hot
import nltk
import re
from nltk.corpus import stopwords


In [18]:
X

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


#### One Hot Representation

In [19]:
X=X.drop('id',axis=1)

In [20]:
X=X.reset_index(drop=True)
df1=df1.reset_index(drop=True)

In [21]:
messages=X.copy()
messages1=df1.copy()

In [22]:
messages1

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...
...,...,...,...,...
5195,25995,The Bangladeshi Traffic Jam That Never Ends - ...,Jody Rosen,Of all the dysfunctions that plague the world’...
5196,25996,John Kasich Signs One Abortion Bill in Ohio bu...,Sheryl Gay Stolberg,WASHINGTON — Gov. John Kasich of Ohio on Tu...
5197,25997,"California Today: What, Exactly, Is in Your Su...",Mike McPhate,Good morning. (Want to get California Today by...
5198,25998,300 US Marines To Be Deployed To Russian Borde...,,« Previous - Next » 300 US Marines To Be Deplo...


In [23]:
## DataPreprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0,len(messages)):
    review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    
    corpus.append(review)

In [24]:
## DataPreprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus_test = []
for i in range(0,len(messages1)):
    review = re.sub('[^a-zA-Z]',' ',messages1['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    
    corpus_test.append(review)

In [25]:
onehot_repr = [one_hot(words,voc_size)for words in corpus]
onehot_repr_test = [one_hot(words,voc_size)for words in corpus_test]

In [26]:
## Embedding
sent_length = 20
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
embedded_docs_test = pad_sequences(onehot_repr_test,padding='pre',maxlen=sent_length)

In [27]:
#MODEL:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)
test_final = np.array(embedded_docs_test)
X_final.shape,y_final.shape
test_final.shape


(5200, 20)

In [29]:
model.fit(X_final, y,epochs=10, batch_size=64, verbose=1)

Epoch 1/10
325/325 [==============================] - 11s 25ms/step - loss: 0.2703 - accuracy: 0.8837
Epoch 2/10
325/325 [==============================] - 8s 26ms/step - loss: 0.1371 - accuracy: 0.9462
Epoch 3/10
325/325 [==============================] - 8s 26ms/step - loss: 0.1067 - accuracy: 0.9599
Epoch 4/10
325/325 [==============================] - 8s 25ms/step - loss: 0.0867 - accuracy: 0.9679
Epoch 5/10
325/325 [==============================] - 8s 26ms/step - loss: 0.0688 - accuracy: 0.9763
Epoch 6/10
325/325 [==============================] - 9s 26ms/step - loss: 0.0498 - accuracy: 0.9831
Epoch 7/10
325/325 [==============================] - 8s 26ms/step - loss: 0.0356 - accuracy: 0.9882
Epoch 8/10
325/325 [==============================] - 8s 25ms/step - loss: 0.0254 - accuracy: 0.9915
Epoch 9/10
325/325 [==============================] - 8s 25ms/step - loss: 0.0186 - accuracy: 0.9942
Epoch 10/10
325/325 [==============================] - 8s 26ms/step - loss: 0.0131 - accur

In [30]:
y_pred = (model.predict(test_final) > 0.5).astype("int32")

163/163 [==============================] - 2s 7ms/step


In [31]:
y_pred

array([[0],
       [1],
       [0],
       ...,
       [0],
       [1],
       [0]], dtype=int32)

In [32]:
final_sub = pd.DataFrame()
final_sub['id']=df1['id']
final_sub['label'] = y_pred
final_sub.to_csv('final_sub.csv',index=False)

In [33]:
final_sub.head()

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,0
4,20804,1
